## Preprocessing

In [1]:
# Import our dependencies
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
%matplotlib inline


In [9]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
wine_df = pd.read_csv('clean_wine_data_final.csv')

# Review the DataFrame
wine_df.head()

,country,description,points,price,province,region,title,variety,winery,rating_category,type,vintage
0,US,"Tart and snappy, the flavors of lime flesh and...",87,14,Oregon,Willamette Valley,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,Good,White,2013
1,US,"Pineapple rind, lemon pith and orange blossom ...",87,13,Michigan,Lake Michigan Shore,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,Good,White,2013
2,US,"Much like the regular bottling from 2012, this...",87,65,Oregon,Willamette Valley,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,Good,Red,2012
3,France,This has great depth of flavor with its fresh ...,87,27,Alsace,Alsace,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam,Good,White,2012
4,US,"Soft, supple plum envelopes an oaky structure ...",87,19,California,Napa Valley,Kirkland Signature 2011 Mountain Cuvée Caberne...,Cabernet Sauvignon,Kirkland Signature,Good,Red,2011


In [10]:
#Review Data Types
wine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77931 entries, 0 to 77930
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   country          77931 non-null  object
 1   description      77931 non-null  object
 2   points           77931 non-null  int64 
 3   price            77931 non-null  int64 
 4   province         77931 non-null  object
 5   region           77931 non-null  object
 6   title            77931 non-null  object
 7   variety          77931 non-null  object
 8   winery           77931 non-null  object
 9   rating_category  77931 non-null  object
 10  type             77931 non-null  object
 11  vintage          77931 non-null  object
dtypes: int64(2), object(10)
memory usage: 7.1+ MB


In [11]:
wine_df['type'].value_counts()

type
Red          53098
White        18962
Sparkling     3032
Rosé          2839
Name: count, dtype: int64

In [12]:
# Drop the non-beneficial ID columns
wine_df = wine_df.drop(columns = ['description','title', 'winery', 'region'])
wine_df.head()

,country,points,price,province,region,variety,rating_category,type,vintage
0,US,87,14,Oregon,Willamette Valley,Pinot Gris,Good,White,2013
1,US,87,13,Michigan,Lake Michigan Shore,Riesling,Good,White,2013
2,US,87,65,Oregon,Willamette Valley,Pinot Noir,Good,Red,2012
3,France,87,27,Alsace,Alsace,Pinot Gris,Good,White,2012
4,US,87,19,California,Napa Valley,Cabernet Sauvignon,Good,Red,2011


In [14]:
#Review range and statistics of points column (range is 80-100, so 90 may make a good boolean).
wine_df['points'].describe()

count    77931.000000
mean        88.601673
std          3.118509
min         80.000000
25%         86.000000
50%         88.000000
75%         91.000000
max        100.000000
Name: points, dtype: float64

In [15]:
wine_df['rating_category'].value_counts()

rating_category
Very Good     26260
Good          25775
Excellent     20501
Acceptable     3699
Superb         1661
Classic          35
Name: count, dtype: int64

In [18]:
# Create 'target' column based on >= 90 points for model 
wine_df['target'] =wine_df['points']>=90
wine_df.tail()

,country,points,price,province,region,variety,rating_category,type,vintage,target
77926,US,90,22,California,Chiles Valley,Zinfandel,Very Good,Red,2011,True
77927,France,90,57,Alsace,Alsace,Pinot Gris,Very Good,White,2010,True
77928,France,90,28,Alsace,Alsace,Pinot Gris,Very Good,White,2013,True
77929,US,90,75,Oregon,Oregon,Pinot Noir,Very Good,Red,2004,True
77930,France,90,32,Alsace,Alsace,Pinot Gris,Very Good,White,2012,True


In [20]:
# Drop points related columns 
wine_df = wine_df.drop(columns = ['points','rating_category'])

KeyError: "['points', 'rating_category'] not found in axis"

In [21]:
#Review Dataframe
wine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77931 entries, 0 to 77930
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   country   77931 non-null  object
 1   price     77931 non-null  int64 
 2   province  77931 non-null  object
 3   region    77931 non-null  object
 4   variety   77931 non-null  object
 5   type      77931 non-null  object
 6   vintage   77931 non-null  object
 7   target    77931 non-null  bool  
dtypes: bool(1), int64(1), object(6)
memory usage: 4.2+ MB


In [22]:
# Determine the number of unique values in each column.
wine_df.nunique()

country        7
price        341
province      63
region      1000
variety       21
type           4
vintage       40
target         2
dtype: int64

In [26]:
# Review countries
wine_df.country.unique()

array(['US', 'France', 'Argentina', 'Italy', 'Australia', 'Spain',
       'Canada'], dtype=object)

In [30]:
# Review province (within US) to see if binning is valuable 
province_counts = wine_df['province'].loc[wine_df.country == 'US'].value_counts()
province_counts

province
California    31456
Washington     7387
Oregon         4821
New York       2252
USOther         897
Name: count, dtype: int64

In [31]:
# Binning attempt for US
provinces_to_replace = province_counts[province_counts < 2000].index

# Replace in dataframe
for prov in provinces_to_replace:
    wine_df['province'] = wine_df['province'].replace(prov, ('US_' +"Other"))

# Check to make sure binning was successful
wine_df['province'].loc[wine_df.country == 'US'].value_counts()

province
California    31456
Washington     7387
Oregon         4821
New York       2252
US_Other        897
Name: count, dtype: int64

In [32]:
# Determined binning not needed for initial model - drop region and province (related to country)
wine_df = wine_df.drop(columns = ['province','region'])

In [33]:
wine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77931 entries, 0 to 77930
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   country  77931 non-null  object
 1   price    77931 non-null  int64 
 2   variety  77931 non-null  object
 3   type     77931 non-null  object
 4   vintage  77931 non-null  object
 5   target   77931 non-null  bool  
dtypes: bool(1), int64(1), object(4)
memory usage: 3.0+ MB


In [38]:
# Review binning options by type ('Red')
red_counts = wine_df['variety'].loc[wine_df.type == 'Red'].value_counts()
red_counts

variety
Pinot Noir                  11693
Cabernet Sauvignon           8284
Red Blend                    7537
Bordeaux-style Red Blend     5123
Syrah                        3617
Zinfandel                    2707
Merlot                       2630
Malbec                       2417
Sangiovese                   2369
Nebbiolo                     2325
Tempranillo                  1773
Rhône-style Red Blend        1376
Cabernet Franc               1247
Name: count, dtype: int64

In [39]:
# Perform binning by type (red) to create "Red_Other"
red_to_replace = red_counts[red_counts < 5000].index

# Replace in dataframe
for red in red_to_replace:
    wine_df['variety'] = wine_df['variety'].replace(red, ('Red_' +"Other"))

# Check to make sure binning was successful
wine_df['variety'].loc[wine_df.type == 'Red'].value_counts()

variety
Red_Other                   20461
Pinot Noir                  11693
Cabernet Sauvignon           8284
Red Blend                    7537
Bordeaux-style Red Blend     5123
Name: count, dtype: int64

In [41]:
# Review white wine bins 
white_counts = wine_df['variety'].loc[wine_df.type == 'White'].value_counts()
white_counts

variety
Chardonnay         10070
Sauvignon Blanc     3177
Riesling            2593
White Blend         1838
Pinot Gris          1284
Name: count, dtype: int64

In [42]:
# Repeat binning for White wines
white_to_replace = white_counts[white_counts < 2600].index

# Replace in dataframe
for variety in white_to_replace:
    wine_df['variety'] = wine_df['variety'].replace(variety, ('White_' +"Other"))

# Check to make sure binning was successful
wine_df['variety'].loc[wine_df.type == 'White'].value_counts()

variety
Chardonnay         10070
White_Other         5715
Sauvignon Blanc     3177
Name: count, dtype: int64

In [44]:
# Review binning overall for variety (down to 11 categories)
wine_df['variety'].value_counts()

variety
Red_Other                   20461
Pinot Noir                  11693
Chardonnay                  10070
Cabernet Sauvignon           8284
Red Blend                    7537
White_Other                  5715
Bordeaux-style Red Blend     5123
Sauvignon Blanc              3177
Rosé                         2839
Sparkling Blend              1837
Champagne Blend              1195
Name: count, dtype: int64

In [43]:
#Review total bins for dataframe
wine_df.nunique()

country      7
price      341
variety     11
type         4
vintage     40
target       2
dtype: int64

In [46]:
# Review vintage bins
vintage_counts = wine_df.vintage.value_counts()
vintage_counts

vintage
2012       10142
2013        9872
2014        9717
2010        7512
2011        7388
2009        6090
2015        5051
2008        4330
2007        3990
2006        3349
No Year     2675
2005        2373
2016        1747
2004        1079
2000         498
1999         441
1998         429
2001         346
2003         296
1997         269
2002         194
1996          45
1995          33
1994          18
1992          11
2017           5
1990           5
1991           4
1988           4
1989           4
1985           3
1887           2
1993           2
1986           1
1919           1
1987           1
1872           1
1904           1
1882           1
1945           1
Name: count, dtype: int64

In [47]:
#Perform binning for vintage
vintage_to_replace = vintage_counts[vintage_counts < 3000].index

# Replace in dataframe
for variety in vintage_to_replace:
    wine_df['vintage'] = wine_df['vintage'].replace(variety, ("Other"))

# Check to make sure binning was successful
wine_df.vintage.value_counts()

vintage
Other    10490
2012     10142
2013      9872
2014      9717
2010      7512
2011      7388
2009      6090
2015      5051
2008      4330
2007      3990
2006      3349
Name: count, dtype: int64

In [48]:
# Review nunique
wine_df.nunique()

country      7
price      341
variety     11
type         4
vintage     11
target       2
dtype: int64

In [49]:
#Review Dataframe 
wine_df.head()

,country,price,variety,type,vintage,target
0,US,14,White_Other,White,2013,False
1,US,13,White_Other,White,2013,False
2,US,65,Pinot Noir,Red,2012,False
3,France,27,White_Other,White,2012,False
4,US,19,Cabernet Sauvignon,Red,2011,False


In [50]:
# Drop type now that binning is complete 
wine_df = wine_df.drop(columns = ['type'])

In [51]:
# Review columns 
wine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77931 entries, 0 to 77930
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   country  77931 non-null  object
 1   price    77931 non-null  int64 
 2   variety  77931 non-null  object
 3   vintage  77931 non-null  object
 4   target   77931 non-null  bool  
dtypes: bool(1), int64(1), object(3)
memory usage: 2.5+ MB


In [52]:
# Convert categorical data to numeric with `pd.get_dummies`
wine_dummies_df = pd.get_dummies(wine_df, dtype = int)
wine_dummies_df.head()

,price,target,country_Argentina,country_Australia,country_Canada,country_France,country_Italy,country_Spain,country_US,variety_Bordeaux-style Red Blend,...,vintage_2007,vintage_2008,vintage_2009,vintage_2010,vintage_2011,vintage_2012,vintage_2013,vintage_2014,vintage_2015,vintage_Other
0,14,False,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
1,13,False,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
2,65,False,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,27,False,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,19,False,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0


In [54]:
#Review dummy columns 
wine_dummies_df.columns

Index(['price', 'target', 'country_Argentina', 'country_Australia',
       'country_Canada', 'country_France', 'country_Italy', 'country_Spain',
       'country_US', 'variety_Bordeaux-style Red Blend',
       'variety_Cabernet Sauvignon', 'variety_Champagne Blend',
       'variety_Chardonnay', 'variety_Pinot Noir', 'variety_Red Blend',
       'variety_Red_Other', 'variety_Rosé', 'variety_Sauvignon Blanc',
       'variety_Sparkling Blend', 'variety_White_Other', 'vintage_2006',
       'vintage_2007', 'vintage_2008', 'vintage_2009', 'vintage_2010',
       'vintage_2011', 'vintage_2012', 'vintage_2013', 'vintage_2014',
       'vintage_2015', 'vintage_Other'],
      dtype='object')

In [55]:
# Drop unecssary column from each variable (country, variety, vintage)
wine_dummies_df= wine_dummies_df.drop(columns = ['variety_White_Other','vintage_Other','country_US'])
wine_dummies_df.head()

,price,target,country_Argentina,country_Australia,country_Canada,country_France,country_Italy,country_Spain,variety_Bordeaux-style Red Blend,variety_Cabernet Sauvignon,...,vintage_2006,vintage_2007,vintage_2008,vintage_2009,vintage_2010,vintage_2011,vintage_2012,vintage_2013,vintage_2014,vintage_2015
0,14,False,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,13,False,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,65,False,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,27,False,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,19,False,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0


In [56]:
# Split our preprocessed data into our features and target arrays
y = wine_dummies_df['target']
X = wine_dummies_df.drop (columns = 'target')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [57]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Random Forest Model

In [60]:
# initiate model instance 
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)


In [61]:
# fit model to training data
rf_model = rf_model.fit(X_train_scaled, y_train)

In [62]:
# Making predictions using the testing data
predictions = rf_model.predict(X_test_scaled)


In [63]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)


In [64]:
# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)


In [65]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))


Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,9406,2236
Actual 1,2920,4921


Accuracy Score : 0.7353590309500591
Classification Report
              precision    recall  f1-score   support

       False       0.76      0.81      0.78     11642
        True       0.69      0.63      0.66      7841

    accuracy                           0.74     19483
   macro avg       0.73      0.72      0.72     19483
weighted avg       0.73      0.74      0.73     19483



## Feature Importance

In [66]:
# extract feature importances 
importances = rf_model.feature_importances_

In [67]:
# display feature importances
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.772149236712036, 'price'),
 (0.01800887911162833, 'country_France'),
 (0.01667684367122795, 'variety_Pinot Noir'),
 (0.013637684400593526, 'country_Italy'),
 (0.011303212479913406, 'variety_Red_Other'),
 (0.010125362440791484, 'variety_Chardonnay'),
 (0.010008633918906296, 'vintage_2013'),
 (0.009978474859319695, 'vintage_2014'),
 (0.009771578051033998, 'country_Spain'),
 (0.00924256089842844, 'country_Argentina'),
 (0.009056287835872714, 'vintage_2012'),
 (0.008721992381504319, 'variety_Bordeaux-style Red Blend'),
 (0.00862062872820746, 'variety_Red Blend'),
 (0.008612307776789847, 'vintage_2011'),
 (0.008565248120772851, 'vintage_2015'),
 (0.008503330303317602, 'vintage_2010'),
 (0.0077540947906158, 'vintage_2008'),
 (0.007717551169640358, 'vintage_2009'),
 (0.0075821284807064595, 'variety_Rosé'),
 (0.0073245124805741526, 'country_Australia'),
 (0.0068709950303801745, 'vintage_2007'),
 (0.006561101788048148, 'variety_Cabernet Sauvignon'),
 (0.006281144863410989, 'vintage_2006'),


In [68]:
# import joblib for saving model
import joblib

In [69]:
# Save model
joblib.dump(rf_model, "my_random_forest.joblib")

['my_random_forest.joblib']

In [70]:
#load model
loaded_model = joblib.load("my_random_forest.joblib")

In [71]:
# test loaded model by creating predictions
predictions2 = loaded_model.predict(X_test_scaled)

In [73]:
# Calculating the confusion matrix
cm2 = confusion_matrix(y_test, predictions2)
cm_df2 = pd.DataFrame(
    cm2, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

In [74]:
# Display confusion matrix 
display(cm_df2)

,Predicted 0,Predicted 1
Actual 0,9406,2236
Actual 1,2920,4921


In [76]:
# Display classificaiton report 
print(classification_report(y_test, predictions2))

              precision    recall  f1-score   support

       False       0.76      0.81      0.78     11642
        True       0.69      0.63      0.66      7841

    accuracy                           0.74     19483
   macro avg       0.73      0.72      0.72     19483
weighted avg       0.73      0.74      0.73     19483

